# Compute topography indices
**Please clean all outputs when committing all the changes in git.**

In [ ]:
%matplotlib inline  
import os, sys
import glob
import xarray as xr
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import scripts.utility as util

print("\nThe Python version: %s.%s.%s" % sys.version_info[:3])

-------------------------
## 1. Setup <a id='setup'></a>

In [ ]:
setup = util.load_yaml("./topo_index_setup.yaml")

catchment_name = setup["catchment_name"]  # Catchment case: CONUS_HUC12, camels, camelsx-v3
serial = setup["serial"] # True: dask distributed enabled
saveCSV = setup["saveCSV"] # True: save attributes for each HRU in csv
saveNetCDF = setup["saveNetCDF"] # True: save attributes for each HRU in netcdf
remap = setup["remap"] # True: remap meteorological time series to HRUs

# files and directories
src_dir = setup["src_dir"]
catch_gpkg = setup["catch_gpkg"]
mapping_file = setup["mapping_file"]

# topography data meta
nc_topo_dict = setup['nc_topo_dict']

# catchment meta
catch_attrs = setup["catch_attrs"]

print('-- Setup:')
print(f" Dask not enabled: {serial}")
print(f" catchment_name: {catchment_name}")
print(f" saveCSV: {saveCSV}")
print(f" saveNetCDF: {saveNetCDF}")
print(f" remap: {remap}")
print(f" topography data directory: {src_dir}")
print(f" catchment gpkg: {catch_gpkg}")
print(f" mapping_file: {mapping_file}")

In [ ]:
%%time
print(f'Reading dem data')
for ix, var in enumerate(nc_topo_dict.keys()):
    print(f'{ix} {var}')
    if ix==0:
        ds = xr.open_dataset(os.path.join(src_dir, f'MERIT_LOCA2_{var}_0.02.nc')).load()
    else:
        ds = ds.merge(xr.open_dataset(os.path.join(src_dir, f'MERIT_LOCA2_{var}_0.02.nc')).load())
ds['slope'] = np.tan(ds['slope']/100)

## Re-mapping
- Elevation [m], slope [m/m]

In [ ]:
%%time
dr_mask = xr.where(np.isnan(ds['elevation']), 0, 1)
a = util.regrid_mean(xr.open_dataset(mapping_file), ds, dr_mask, ['elevation', 'slope', 'eastness', 'northness'])
a

## Dataset to Dataframe

In [ ]:
df = a.to_dataframe()

## Save in csv or netcdf

In [ ]:
if saveCSV:
    df.to_csv(os.path.join('output', f'{catchment_name}_topo.csv'), float_format='%g')
if saveNetCDF:
    a.to_netcdf(os.path.join('output', f'{catchment_name}_topo.nc'))

## Plotting

In [ ]:
gdf_camels = util.read_shps([catch_gpkg], [catch_attrs[catchment_name]['id']])

In [ ]:
gdf_camels = gdf_camels.merge(df,left_on=catch_attrs[catchment_name]['id'], right_index=True)

In [ ]:
var_name = 'elevation'
fig, ax = plt.subplots(figsize=(6, 3), dpi=150)
gdf_camels.plot(ax=ax, column=var_name, cmap='turbo',
                norm=colors.Normalize(0, 3000),
                legend=True
               );
ax.set_title(var_name);
fig.tight_layout()
plt.savefig(f'./figures/{catchment_name}_topo_{var_name}.png', dpi=300)

In [ ]:
var_name = 'slope'
fig, ax = plt.subplots(figsize=(6, 3), dpi=150)
gdf_camels.plot(ax=ax, column=var_name, cmap='turbo',
                norm=colors.Normalize(0, 0.1),
                legend=True
               );
ax.set_title(var_name);
fig.tight_layout()
plt.savefig(f'./figures/{catchment_name}_topo_{var_name}.png', dpi=300)

In [ ]:
var_name = 'northness'
fig, ax = plt.subplots(figsize=(6, 3), dpi=150)
gdf_camels.plot(ax=ax, column=var_name, cmap='turbo',
                norm=colors.Normalize(-0.4, 0.4),
                legend=True
               );
ax.set_title(var_name);
fig.tight_layout()
plt.savefig(f'./figures/{catchment_name}_topo_{var_name}.png', dpi=300)

In [ ]:
var_name = 'eastness'
fig, ax = plt.subplots(figsize=(6, 3), dpi=150)
gdf_camels.plot(ax=ax, column=var_name, cmap='turbo',
                norm=colors.Normalize(-0.4, 0.4),
                legend=True
               );
ax.set_title(var_name);
fig.tight_layout()
plt.savefig(f'./figures/{catchment_name}_topo_{var_name}.png', dpi=300)